In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv('paradox.csv',sep= ',')
df.head()

,Site,Code,SAincl,SA2,SA4,SA6,SA8,SA10,SA12,SA1416,...,TempPara5,VitD,VitD3Groups,ThreeUTR,ThreeUTRvar,INT4,INT4var,D543,Callele,Callelevar
0,1.0,401,8.3,1.0,2.8,3.3,1.2,1.1,1.1,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,402,11.3,13.3,18.9,20.1,26.2,34.7,29.9,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,403,15.4,9.9,9.5,9.5,8.3,8.7,6.4,3.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,404,0.5,0.2,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,405,33.0,24.1,12.3,11.8,8.7,3.3,3.1,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 49 columns):
Site                     241 non-null float64
Code                     241 non-null int64
SAincl                   241 non-null float64
SA2                      233 non-null float64
SA4                      227 non-null float64
SA6                      223 non-null float64
SA8                      232 non-null float64
SA10                     215 non-null float64
SA12                     225 non-null float64
SA1416                   223 non-null float64
SA2120                   221 non-null float64
SA2728                   211 non-null float64
studyarm                 241 non-null float64
sexe                     241 non-null float64
age                      241 non-null int64
lesionsince              240 non-null float64
BPSYST                   216 non-null float64
BPDIAST                  216 non-null float64
pulserateinclbeatsmin    232 non-null float64
tempinclCelsius          

## Data Cleaning

In [5]:
target = 'Paradox1'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
#df

### Remove outliers

In [7]:
# Drop the column `Code` as it is unrelated to the dependent variable
df = df.drop(['Code'],axis=1)
df.head()

,Site,SAincl,SA2,SA4,SA6,SA8,SA10,SA12,SA1416,SA2120,...,TempPara5,VitD,VitD3Groups,ThreeUTR,ThreeUTRvar,INT4,INT4var,D543,Callele,Callelevar
0,1.0,8.3,1.0,2.8,3.3,1.2,1.1,1.1,1.1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,11.3,13.3,18.9,20.1,26.2,34.7,29.9,33.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,15.4,9.9,9.5,9.5,8.3,8.7,6.4,3.9,3.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,0.5,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,33.0,24.1,12.3,11.8,8.7,3.3,3.1,3.1,7.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Remove bad columns

In [8]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [9]:
df = remove_bad_columns(df,40)
df.info()

number of bad columns: 15
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 33 columns):
Site                     241 non-null float64
SAincl                   241 non-null float64
SA2                      233 non-null float64
SA4                      227 non-null float64
SA6                      223 non-null float64
SA8                      232 non-null float64
SA10                     215 non-null float64
SA12                     225 non-null float64
SA1416                   223 non-null float64
SA2120                   221 non-null float64
SA2728                   211 non-null float64
studyarm                 241 non-null float64
sexe                     241 non-null float64
age                      241 non-null int64
lesionsince              240 non-null float64
BPSYST                   216 non-null float64
BPDIAST                  216 non-null float64
pulserateinclbeatsmin    232 non-null float64
tempinclCelsius          237 non-null float

### Remove bad rows

In [10]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [11]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [12]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [13]:
bad_rows = find_bad_rows(df,5)
df.info()

number of bad rows: 12
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 33 columns):
Site                     241 non-null float64
SAincl                   241 non-null float64
SA2                      233 non-null float64
SA4                      227 non-null float64
SA6                      223 non-null float64
SA8                      232 non-null float64
SA10                     215 non-null float64
SA12                     225 non-null float64
SA1416                   223 non-null float64
SA2120                   221 non-null float64
SA2728                   211 non-null float64
studyarm                 241 non-null float64
sexe                     241 non-null float64
age                      241 non-null int64
lesionsince              240 non-null float64
BPSYST                   216 non-null float64
BPDIAST                  216 non-null float64
pulserateinclbeatsmin    232 non-null float64
tempinclCelsius          237 non-null float64


In [14]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 12


### Separate target and attributes

In [15]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [16]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[  2 177 155 151 121 129 117 121 112  74  46   4   2  41  21  13  12  75
  33  59  78   3   5   3   4 119 106  81   2   2   2   8]


In [17]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [18]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [0, 12, 28, 29, 30]
i_category: [11, 21, 23, 24]
variable type: [1. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 2. 1. 3. 3. 3. 3. 3. 3. 3. 3. 2. 3. 2.
 2. 3. 3. 3. 1. 1. 1. 3.]


### Impute missing values of attributes

In [19]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [20]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229 entries, 0 to 240
Data columns (total 32 columns):
Site                     229 non-null float64
SAincl                   229 non-null float64
SA2                      229 non-null float64
SA4                      229 non-null float64
SA6                      229 non-null float64
SA8                      229 non-null float64
SA10                     229 non-null float64
SA12                     229 non-null float64
SA1416                   229 non-null float64
SA2120                   229 non-null float64
SA2728                   229 non-null float64
studyarm                 229 non-null float64
sexe                     229 non-null float64
age                      229 non-null int64
lesionsince              229 non-null float64
BPSYST                   229 non-null float64
BPDIAST                  229 non-null float64
pulserateinclbeatsmin    229 non-null float64
tempinclCelsius          229 non-null float64
bodyweightinclkg       

## Data Processing

### Attributes

In [21]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [22]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(229, 42)
[[-1.   8.3  1.  ... -1.   1.   6. ]
 [-1.  11.3 13.3 ... -1.  -1.   0. ]
 [-1.  15.4  9.9 ... -1.  -1.   0. ]
 ...
 [ 1.  38.4 10.6 ... -1.  -1.   0. ]
 [ 1.  68.1 79.2 ... -1.   1.  10. ]
 [ 1.   4.   1.5 ...  1.  -1.   0. ]]


### Target

In [23]:
y = np.array(dfy)
#print(np.unique(y,return_counts=True))

# convert taget to 0 and 1
y_new = y
#y_new = np.ones(y.shape[0])
#y_new[y =='No'] = 0

print(np.unique(y_new,return_counts=True))

(array([0., 1.]), array([169,  60]))


In [24]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('data_processed.dat',xy_new,fmt='%f')